In [30]:
import polars as pl
pl.Config.set_fmt_str_lengths(200)

import requests as rq
import json

### Keys

In [2]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [16]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

### API status

In [3]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [4]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [5]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

In [6]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

#### List of coins

In [7]:
def get_list_of_coins():

    response = get_response("/coins/list", use_demo, "", PUB_URL)

    return pl.DataFrame(response) 

In [8]:
df_coins = get_list_of_coins()

In [9]:
df_coins.filter(pl.col("symbol") == "btc")

id,symbol,name
str,str,str
"""batcat""","""btc""","""batcat"""
"""bitcoin""","""btc""","""Bitcoin"""
"""blackrocktradingcurrency""","""btc""","""BlackrockTradingCurrency"""
"""bobby-the-cat""","""btc""","""Bobby The Cat"""
"""osmosis-allbtc""","""btc""","""Osmosis allBTC"""


### OHLC

In [10]:
def get_ohlc_data(coin_id, target_curr, days, precision):

    ohlc_params = {
            "vs_currency": target_curr,
            "days": days,
            "precision": precision
    }

    ohlc_list_response = get_response(f"/coins/{coin_id}/ohlc",
                                      use_demo,
                                      ohlc_params,
                                      PUB_URL)

    df_ohlc = pl.DataFrame(ohlc_list_response, 
                           schema = ["timestamp", "open", "high", "low", "close"],
                           orient = "row")

    df_ohlc = df_ohlc.with_columns(
                  pl.from_epoch(pl.col("timestamp"),
                                time_unit = "ms").alias("timestamp")
         )

    return df_ohlc

In [11]:
df_ohlc_btc = get_ohlc_data("bitcoin", "eur", "14", "2")

df_ohlc_btc

timestamp,open,high,low,close
datetime[ms],f64,f64,f64,f64
2025-07-01 20:00:00,90003.37,90303.59,89532.34,89532.34
2025-07-02 00:00:00,89374.75,89819.18,89370.19,89468.81
2025-07-02 04:00:00,89512.11,90003.85,89268.68,90002.3
2025-07-02 08:00:00,89990.28,91003.04,89990.13,91003.04
2025-07-02 12:00:00,91007.33,91638.41,91007.33,91374.54
…,…,…,…,…
2025-07-15 00:00:00,102741.96,103139.28,102616.75,102728.14
2025-07-15 04:00:00,102738.77,102738.77,100364.29,100465.0
2025-07-15 08:00:00,100320.25,100721.95,99822.22,99860.19


#### Verify column assignment

In [12]:
def validate_columns(df):    

    # Create expressions for max and min across the OHLC columns
    validation = df.with_columns([
        pl.max_horizontal(["open", "high", "low", "close"]).alias("row_max"),
        pl.min_horizontal(["open", "high", "low", "close"]).alias("row_min")
    ])

    not_valid = (
        (validation["high"] != validation["row_max"]) &
        (validation["low"] != validation["row_min"])
    ).all()
    
    print("Invalid OHLC rows:", not_valid)

    return None

In [13]:
validate_columns(df_ohlc_btc)

Invalid OHLC rows: False


#### Export to CSV / XLSX

In [14]:
df_export = get_ohlc_data("bitcoin", "eur", "14", "2")

df_export.write_csv("/home/vikas/Desktop/GitHub/CoinGecko_export_data/BTC_EUR_export.csv")
df_export.write_excel("/home/vikas/Desktop/GitHub/CoinGecko_export_data/BTC_EUR_export.xlsx",
                      autofit = True)

### OHLCV

#### Get top pools for a specific network

In [17]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [24]:
def get_url(url_type,
            network,
            dex = "",
            pool_address = "",
            token_address = ""):   

    url_dict = {
        "trending_pools": f"/onchain/networks/{network}/trending_pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}",
        "top_pools_add": f"/onchain/networks/{network}/tokens/{token_address}/pools"
    }

    return url_dict[url_type]

def collect_response(list_response):    

    response_all = []

    for response in list_response["data"]:
        all_attributes = response["attributes"]
        daily_tx = all_attributes["transactions"]["h24"]
        rel = response["relationships"]
        
        temp_dict = dict(
            pair = all_attributes["name"],
            dex = rel["dex"]["data"]["id"],
            add = all_attributes["address"],
            fdv_usd = all_attributes["fdv_usd"],
            market_cap_usd = all_attributes["market_cap_usd"],
            daily_volume = all_attributes["volume_usd"]["h24"],
            daily_price_change = all_attributes["price_change_percentage"]["h24"],
            daily_buys = daily_tx["buys"],
            daily_sells = daily_tx["sells"],
            daily_buyers = daily_tx["buyers"],
            daily_sellers = daily_tx["sellers"]
        )
        
        response_all.append(temp_dict)

    return response_all

def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pl.DataFrame(toppool_all).sort([f"{sort_by_col}"], descending = True)

In [32]:
get_top_pools_network("eth", "daily_volume").limit(5)

pair,dex,add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
str,str,str,str,str,str,str,i64,i64,i64,i64
"""EDOGE / WETH""","""uniswap_v2""","""0x693e6c4ac1ab7203111f78d3c53d4083a1e963ab""","""8032.9090123467""",null,"""979647.988340449""","""102.35""",1731,1535,886,561
"""HuaHua / WETH""","""uniswap_v2""","""0x216f74bf5ede14e271b659d9e6ba75ee594032d5""","""193895.147030122""",null,"""902851.477555977""","""28.86""",1210,1062,582,475
"""Rudi / WETH""","""uniswap_v2""","""0x70aae7f70b3f21df3aaccb4fabc669630d32f307""","""103308.501864737""",null,"""874328.394646964""","""80.06""",1352,984,662,496
"""MANYU / WETH""","""uniswap_v2""","""0xc4704f13d5e08b27b039d53873e813dd2fad99d9""","""25766724.1909438""","""26052624.2""","""7646208.93177283""","""-25.59""",3258,1971,1845,1216
"""CHAD / WETH""","""uniswap_v2""","""0x2589d559baa45511195ed5c89c0447565d5532ed""","""197513.277760348""",null,"""746693.321693132""","""-60.87""",932,765,493,495


#### OHLCV for a specific pool

In [52]:
def get_ohlcv_data(network, pool_add, timeframe):

    ohlcv_params = ""

    ohlcv_list_response = get_response(f"/onchain/networks/{network}/pools/{pool_address}/ohlcv/{timeframe}",
                                       use_pro,
                                       ohlcv_params,
                                       PRO_URL)

    df_ohlcv = pl.DataFrame(ohlcv_list_response["data"]["attributes"]["ohlcv_list"], 
                           schema = ["timestamp", 
                                     "open", 
                                     "high",
                                     "low",
                                     "close",
                                     "volume"],
                           orient = "row")

    df_ohlcv = df_ohlcv.with_columns(
                  pl.from_epoch(pl.col("timestamp"),
                                time_unit = "s").alias("timestamp")
         )

    return df_ohlcv

In [57]:
network = "eth"
pool_address = "0x693e6c4ac1ab7203111f78d3c53d4083a1e963ab"
timeframe = "minute?aggregate=5"

df_ohlcv = get_ohlcv_data(network, pool_address, timeframe)

df_ohlcv

timestamp,open,high,low,close,volume
datetime[μs],f64,f64,f64,f64,f64
2025-07-15 20:05:00,1.9114e-7,1.9721e-7,1.9114e-7,1.9721e-7,138.391189
2025-07-15 20:00:00,1.8965e-7,1.9114e-7,1.8965e-7,1.9114e-7,17.85991
2025-07-15 19:55:00,1.8932e-7,1.8965e-7,1.8932e-7,1.8965e-7,4.943187
2025-07-15 19:50:00,1.9270e-7,1.9270e-7,1.8811e-7,1.8932e-7,41.483494
2025-07-15 19:45:00,1.9915e-7,1.9915e-7,1.9270e-7,1.9270e-7,20.49023
…,…,…,…,…,…
2025-07-15 11:05:00,0.000002,0.000002,9.2882e-7,0.000001,9809.63409
2025-07-15 11:00:00,0.000004,0.000004,0.000002,0.000002,14151.078195
2025-07-15 10:55:00,0.000006,0.000007,0.000004,0.000004,14732.952273


#### Export to CSV / XLSX

In [58]:
df_ohlcv.write_csv("/home/vikas/Desktop/GitHub/CoinGecko_export_data/OHLCV_export.csv")
df_ohlcv.write_excel("/home/vikas/Desktop/GitHub/CoinGecko_export_data/OHLCV_export.xlsx",
                      autofit = True)